#**Credit Risk Modelling**

Importing necessary libraries

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from sklearn.metrics import accuracy_score,r2_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score
import warnings
import os
import time

In [50]:
print('Program is running...')
print()
start_time=time.time()

Program is running...



Load the dataset

In [51]:
data_1=pd.read_csv('/content/case_study1.xlsx - case_study1.csv')
data_2=pd.read_csv('/content/case_study2.xlsx - case_study2.csv')

In [52]:
data_1.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32


In [53]:
data_2.head()

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1


In [54]:
data_1.describe()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
count,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,...,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000
mean,25668.500000,4.858598,2.770415,2.088184,0.736754,0.428919,0.184574,0.089095,0.577542,0.422458,...,0.124981,1.136084,1.561847,0.070146,0.282511,2.844904,2.013694,1.089762,-32.575639,-62.149525
std,14819.571046,7.177116,5.941680,2.290774,1.296717,0.989972,0.297414,0.205635,0.379867,0.379867,...,0.505201,2.227997,5.376434,0.340861,0.858168,6.187177,3.198322,2.417496,2791.869609,2790.818622
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-99999.000000,-99999.000000
25%,12834.750000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,4.000000
50%,25668.500000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.556000,0.444000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,33.000000,8.000000
75%,38502.250000,5.000000,3.000000,3.000000,1.000000,1.000000,0.308000,0.053000,1.000000,0.750000,...,0.000000,1.000000,1.000000,0.000000,0.000000,3.000000,2.000000,1.000000,64.000000,17.000000
max,51336.000000,235.000000,216.000000,47.000000,27.000000,19.000000,1.000000,1.000000,1.000000,1.000000,...,27.000000,41.000000,235.000000,10.000000,29.000000,235.000000,55.000000,80.000000,392.000000,392.000000


In [55]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [56]:
#remove nulls (those rows are removed in which age_oldest_tl is -99999)
data_1=data_1.loc[data_1['Age_Oldest_TL']!=-99999]

In [57]:
data_1.shape

(51296, 26)

In [58]:
#now for deleting the null values for data_2
col_rem=[]
for i in data_2.columns:
  if data_2.loc[data_2[i]==-99999].shape[0]>10000:
    col_rem.append(i)

In [59]:
col_rem

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [60]:
data_2=data_2.drop(columns=col_rem,axis=1)

In [61]:
data_2.shape

(51336, 54)

In [62]:
for i in data_2.columns:
  data_2=data_2.loc[data_2[i]!=-99999]

In [63]:
data_2.shape

(42066, 54)

In [64]:
data_2.isnull().sum()

PROSPECTID                    0
time_since_recent_payment     0
num_times_delinquent          0
max_recent_level_of_deliq     0
num_deliq_6mts                0
num_deliq_12mts               0
num_deliq_6_12mts             0
num_times_30p_dpd             0
num_times_60p_dpd             0
num_std                       0
num_std_6mts                  0
num_std_12mts                 0
num_sub                       0
num_sub_6mts                  0
num_sub_12mts                 0
num_dbt                       0
num_dbt_6mts                  0
num_dbt_12mts                 0
num_lss                       0
num_lss_6mts                  0
num_lss_12mts                 0
recent_level_of_deliq         0
tot_enq                       0
CC_enq                        0
CC_enq_L6m                    0
CC_enq_L12m                   0
PL_enq                        0
PL_enq_L6m                    0
PL_enq_L12m                   0
time_since_recent_enq         0
enq_L12m                      0
enq_L6m 

In [65]:
#checking common column names:
for i in list(data_1.columns):
  if i in list(data_2.columns):
    print(i)

PROSPECTID


In [66]:
#merge the two dataframes, inner join so that no nulls are present
data=pd.merge(data_1,data_2,how='inner',left_on=['PROSPECTID'] ,right_on=['PROSPECTID'])

In [67]:
data

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.00,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
42060,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,others,others,702,P1
42061,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,1.0,0.0,1.000,0.0,0,0,ConsumerLoan,others,661,P3
42062,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,others,686,P2


In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [69]:
data.isna().sum().sum()

0

In [70]:
#Separating the type of features:Categorical | Numerical
for i in data.columns:
  if data[i].dtype=='object':
    print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [71]:
#chi-square test
for i in ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2','Approved_Flag']:
  chi2,pval,_,_ = chi2_contingency(pd.crosstab(data[i],data['Approved_Flag']))
  print(i,'---',pval)

#since all the categorical features have pval <= 0.05, we will accept all

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287
Approved_Flag --- 0.0


In [72]:
#VIF for numerical columns
num_cols=[]
for i in data.columns:
  if data[i].dtype!='object' and i not in ['PROSPECTID','Approved_flag']:
    num_cols.append(i)

num_cols

['Total_TL',
 'Tot_Closed_TL',
 'Tot_Active_TL',
 'Total_TL_opened_L6M',
 'Tot_TL_closed_L6M',
 'pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'pct_active_tl',
 'pct_closed_tl',
 'Total_TL_opened_L12M',
 'Tot_TL_closed_L12M',
 'pct_tl_open_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'Auto_TL',
 'CC_TL',
 'Consumer_TL',
 'Gold_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'num_times_delinquent',
 'max_recent_level_of_deliq',
 'num_deliq_6mts',
 'num_deliq_12mts',
 'num_deliq_6_12mts',
 'num_times_30p_dpd',
 'num_times_60p_dpd',
 'num_std',
 'num_std_6mts',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_6mts',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_6mts',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',

In [73]:
#Multicollinearity vs Correlation
#Predictability of each features by other features
#Correlation is specific to linear relationships between columns(convex function-cannot say whether positively or negatively correlated)

In [74]:
#VIF sequwntially check
vlf_data=data[num_cols]
total_col=vlf_data.shape[1]
col_kept=[]
col_index=0

for i in range(total_col):
  vif_value=variance_inflation_factor(vlf_data.values,col_index)
  print(col_index,'---',vif_value)
  if vif_value<=6:
    col_kept.append(num_cols[i])
    col_index+=1
  else:
    vif_data=vlf_data.drop(columns=num_cols[i],axis=1)



  #72 numerical features-->39 after vif

0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf
0 --- inf


In [75]:
#Check Anova for columns_to_be_kept
from scipy.stats import f_oneway
col_to_be_kept_numerical=[]
for i in col_kept:
  a=list(data[i])
  b=list(data['Approved_Flag'])
  group_P1=[value for value,group in zip(a,b) if group=='P1']
  group_P2=[value for value,group in zip(a,b) if group=='P2']
  group_P3=[value for value,group in zip(a,b) if group=='P3']
  group_P4=[value for value,group in zip(a,b) if group=='P4']

  f_statistict,p_value=f_oneway(group_P1,group_P2,group_P3,group_P4)
  if p_value<=0.05:
    col_to_be_kept_numerical.append(i)


In [76]:
col_to_be_kept_numerical

[]

In [77]:
#feature selection is done for cat and num features
# Label encoding for the categorical features
['MARITALSTATUS', 'EDUCATION', 'GENDER' , 'last_prod_enq2' ,'first_prod_enq2']

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [78]:
data['MARITALSTATUS'].unique()
data['EDUCATION'].unique()
data['GENDER'].unique()
data['last_prod_enq2'].unique()
data['first_prod_enq2'].unique()

array(['PL', 'ConsumerLoan', 'others', 'AL', 'HL', 'CC'], dtype=object)

In [79]:
# Ordinal feature -- EDUCATION
# SSC            : 1
# 12TH           : 2
# GRADUATE       : 3
# UNDER GRADUATE : 3
# POST-GRADUATE  : 4
# OTHERS         : 1
# PROFESSIONAL   : 3


# Others has to be verified by the business end user

In [80]:
data.loc[data['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
data.loc[data['EDUCATION'] == '12TH',['EDUCATION']]             = 2
data.loc[data['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
data.loc[data['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
data.loc[data['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
data.loc[data['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
data.loc[data['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3

In [81]:
data['EDUCATION'].value_counts()
data['EDUCATION'] = data['EDUCATION'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [82]:
data_encoded = pd.get_dummies(data, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])

In [83]:

data_encoded.info()
k = data_encoded.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 91 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    42064 non-null  int64  
 1   Total_TL                      42064 non-null  int64  
 2   Tot_Closed_TL                 42064 non-null  int64  
 3   Tot_Active_TL                 42064 non-null  int64  
 4   Total_TL_opened_L6M           42064 non-null  int64  
 5   Tot_TL_closed_L6M             42064 non-null  int64  
 6   pct_tl_open_L6M               42064 non-null  float64
 7   pct_tl_closed_L6M             42064 non-null  float64
 8   pct_active_tl                 42064 non-null  float64
 9   pct_closed_tl                 42064 non-null  float64
 10  Total_TL_opened_L12M          42064 non-null  int64  
 11  Tot_TL_closed_L12M            42064 non-null  int64  
 12  pct_tl_open_L12M              42064 non-null  float64
 13  p

In [84]:
data_encoded

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,False,False,True,False,False,False,False,False,True,False
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,True,False,False,False,False,False,True,False,False,False
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,True,False,False,False,False,False,False,False,False,True
3,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,False,False,False,False,True,False,False,False,False,False
4,6,6,5,1,0,0,0.000,0.00,0.167,0.833,...,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,True,False,False,False,False,False,True,False,False,False
42060,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,False,False,False,True,False,False,False,False,False,True
42061,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,True,False,False,False,False,False,False,False,False,True
42062,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,True,False,False,False,False,False,False,False,False,True


In [85]:
#Machine learning model fitting

In [86]:
y = data_encoded['Approved_Flag']
x = data_encoded. drop ( ['Approved_Flag'], axis = 1 )


In [87]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [88]:
rf_classifier = RandomForestClassifier(n_estimators = 200, random_state=42)

In [89]:
rf_classifier.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [90]:
y_pred = rf_classifier.predict(x_test)

In [91]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Assuming y_test and y_pred are defined and contain the actual and predicted labels, respectively.
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}\n')

# Calculate precision, recall, and F1 score for each class
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None)

# Print precision, recall, and F1 score for each class
for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.9900154522762391

Class p1:
Precision: 0.9465290806754222
Recall: 0.995069033530572
F1 Score: 0.9701923076923077

Class p2:
Precision: 0.9954617205998422
Recall: 1.0
F1 Score: 0.9977256995945812

Class p3:
Precision: 0.9968102073365231
Recall: 0.9433962264150944
F1 Score: 0.9693679720822024

Class p4:
Precision: 1.0
Recall: 0.9961127308066083
F1 Score: 0.9980525803310614



In [92]:
# 2. xgboost

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)



y = data_encoded['Approved_Flag']
x = data_encoded. drop ( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)




xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 1.00

Class p1:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Class p2:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Class p3:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Class p4:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0



In [93]:
# 3. Decision Tree
from sklearn.tree import DecisionTreeClassifier


y = data_encoded['Approved_Flag']
x = data_encoded. drop ( ['Approved_Flag'], axis = 1 )

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 1.00

Class p1:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Class p2:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Class p3:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Class p4:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0



In [94]:
#Xg boost is giving the highest accuracy so we will pick it and finetune it

In [95]:
# Hyperparameter tuning in xgboost
from sklearn.model_selection import GridSearchCV
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

# Define the XGBClassifier with the initial set of hyperparameters
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

# Define the parameter grid for hyperparameter tuning

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}


In [96]:
# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test, y_test)
print("Test Accuracy:", accuracy)

# Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}


# Based on risk appetite of the bank, you will suggest P1,P2,P3,P4 to the business end user

Test Accuracy: 1.0


In [97]:
# # Hyperparameter tuning for xgboost (Used in the session)

# # Define the hyperparameter grid
param_grid = {
   'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
   'learning_rate'   : [0.001, 0.01, 0.1, 1],
   'max_depth'       : [3, 5, 8, 10],
   'alpha'           : [1, 10, 100],
   'n_estimators'    : [10,50,100]
 }

index = 0

answers_grid = {
     'combination'       :[],
     'train_Accuracy'    :[],
     'test_Accuracy'     :[],
     'colsample_bytree'  :[],
     'learning_rate'     :[],
     'max_depth'         :[],
     'alpha'             :[],
     'n_estimators'      :[]

     }



In [99]:
# # Loop through each combination of hyperparameters
for colsample_bytree in param_grid['colsample_bytree']:
   for learning_rate in param_grid['learning_rate']:
     for max_depth in param_grid['max_depth']:
       for alpha in param_grid['alpha']:
           for n_estimators in param_grid['n_estimators']:

               index = index + 1

               # Define and train the XGBoost model
               model = xgb.XGBClassifier(objective='multi:softmax',
                                        num_class=4,
                                        colsample_bytree = colsample_bytree,
                                        learning_rate = learning_rate,
                                        max_depth = max_depth,
                                        alpha = alpha,
                                        n_estimators = n_estimators)



               y = data_encoded['Approved_Flag']
               x = data_encoded. drop ( ['Approved_Flag'], axis = 1 )

               label_encoder = LabelEncoder()
               y_encoded = label_encoder.fit_transform(y)


               x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)


               model.fit(x_train, y_train)



               # Predict on training and testing sets
               y_pred_train = model.predict(x_train)
               y_pred_test = model.predict(x_test)


               # Calculate train and test results

               train_accuracy =  accuracy_score (y_train, y_pred_train)
               test_accuracy  =  accuracy_score (y_test , y_pred_test)



               # Include into the lists
               answers_grid ['combination']   .append(index)
               answers_grid ['train_Accuracy']    .append(train_accuracy)
               answers_grid ['test_Accuracy']     .append(test_accuracy)
               answers_grid ['colsample_bytree']   .append(colsample_bytree)
               answers_grid ['learning_rate']      .append(learning_rate)
               answers_grid ['max_depth']          .append(max_depth)
               answers_grid ['alpha']              .append(alpha)
               answers_grid ['n_estimators']       .append(n_estimators)


               # Print results for this combination
               print(f"Combination {index}")
               print(f"colsample_bytree: {colsample_bytree}, learning_rate: {learning_rate}, max_depth: {max_depth}, alpha: {alpha}, n_estimators: {n_estimators}")
               print(f"Train Accuracy: {train_accuracy:.2f}")
               print(f"Test Accuracy : {test_accuracy :.2f}")
               print("-" * 30)


Combination 721
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 10
Train Accuracy: 0.75
Test Accuracy : 0.74
------------------------------
Combination 722
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 50
Train Accuracy: 0.67
Test Accuracy : 0.66
------------------------------
Combination 723
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 100
Train Accuracy: 0.67
Test Accuracy : 0.66
------------------------------
Combination 724
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 10
Train Accuracy: 0.75
Test Accuracy : 0.74
------------------------------
Combination 725
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 50
Train Accuracy: 0.66
Test Accuracy : 0.66
------------------------------
Combination 726
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 100
Train Accuracy: 0.

KeyboardInterrupt: 

In [100]:
a3=pd.read_excel('/content/Unseen_Dataset.xlsx')

In [103]:
cols_in_df=list(data.columns)
cols_in_df.pop(42)


'num_dbt_12mts'

In [106]:
# List of columns to be used
cols_in_df = ['PROSPECTID', 'Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL', 'Total_TL_opened_L6M',
              'Tot_TL_closed_L6M', 'pct_active_tl', 'pct_closed_tl', 'Total_TL_opened_L12M',
              'pct_tl_open_L12M', 'Auto_TL', 'Consumer_TL', 'Gold_TL', 'num_times_delinquent',
              'num_deliq_6mts', 'num_deliq_12mts', 'num_times_30p_dpd', 'num_std', 'num_std_6mts',
              'num_dbt_6mts', 'num_lss_6mts', 'num_lss_12mts', 'tot_enq', 'CC_enq', 'CC_enq_L6m',
              'PL_enq', 'PL_enq_L6m', 'enq_L12m', 'enq_L6m', 'AGE', 'pct_of_active_TLs_ever',
              'pct_opened_TLs_L6m_of_L12m', 'pct_currentBal_all_TL', 'pct_PL_enq_L6m_of_L12m',
              'pct_CC_enq_L6m_of_L12m', 'Credit_Score', 'Approved_Flag']

# Print the columns that are actually in a3
print("Columns in a3 DataFrame:")
print(a3.columns.tolist())

# Find the columns that are missing
missing_cols = [col for col in cols_in_df if col not in a3.columns]
if missing_cols:
    print(f"These columns are missing in a3: {missing_cols}")
else:
    print("All columns are present in a3.")

# Only select the columns that are present in a3
cols_in_df_present = [col for col in cols_in_df if col in a3.columns]

# Select the data with the available columns
df_unseen = a3[cols_in_df_present]

print("DataFrame with selected columns:")
print(df_unseen.head())


Columns in a3 DataFrame:
['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment', 'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd', 'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt', 'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever', 'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
These columns are missing in a3: ['PROSPECTID', 'Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL', 'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_active_tl', 'pct_closed_tl', 'Total_TL_opened_L12M', 'pct_tl_open_L12M', 'Auto_TL', 'Consumer_TL', 'Gold_TL', 'num_t

In [108]:
# Check which columns are present in df_unseen
columns_to_check = ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

# Print available columns in df_unseen
print("Columns in df_unseen:")
print(df_unseen.columns.tolist())

# Check for each column and print unique values if the column exists
for col in columns_to_check:
    if col in df_unseen.columns:
        unique_values = df_unseen[col].unique()
        print(f"Unique values in {col}: {unique_values}")
    else:
        print(f"Column {col} not found in df_unseen.")


Columns in df_unseen:
[]
Column MARITALSTATUS not found in df_unseen.
Column EDUCATION not found in df_unseen.
Column GENDER not found in df_unseen.
Column last_prod_enq2 not found in df_unseen.
Column first_prod_enq2 not found in df_unseen.


In [111]:
import pandas as pd

# Define the path to your Excel file
file_path = '/content/Unseen_Dataset.xlsx'

# Read the Excel file
df_unseen = pd.read_excel(file_path)

# Print the columns in df_unseen
print("Columns in df_unseen:")
print(df_unseen.columns.tolist())

# Check for the presence of required columns and print unique values if the column exists
required_columns = ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

for col in required_columns:
    if col in df_unseen.columns:
        unique_values = df_unseen[col].unique()
        print(f"Unique values in {col}: {unique_values}")
    else:
        print(f"Column {col} not found in df_unseen.")

# Transform 'EDUCATION' column if it exists
if 'EDUCATION' in df_unseen.columns:
    education_mapping = {
        'SSC': 1,
        '12TH': 2,
        'GRADUATE': 3,
        'UNDER GRADUATE': 3,
        'POST-GRADUATE': 4,
        'OTHERS': 1,
        'PROFESSIONAL': 3
    }
    df_unseen['EDUCATION'] = df_unseen['EDUCATION'].map(education_mapping)
    print("Transformed 'EDUCATION' column:")
    print(df_unseen['EDUCATION'].unique())
else:
    print("Column 'EDUCATION' not found in df_unseen.")

# Example: Check for 'MARITALSTATUS' column and perform a similar transformation if needed
# if 'MARITALSTATUS' in df_unseen.columns:
#     marital_status_mapping = {
#         'Single': 1,
#         'Married': 2,
#         'Divorced': 3,
#         'Widowed': 4
#     }
#     df_unseen['MARITALSTATUS'] = df_unseen['MARITALSTATUS'].map(marital_status_mapping)
#     print("Transformed 'MARITALSTATUS' column:")
#     print(df_unseen['MARITALSTATUS'].unique())
# else:
#     print("Column 'MARITALSTATUS' not found in df_unseen.")

# Continue this pattern for other columns that need transformation
# ...

print("DataFrame with transformed columns:")
print(df_unseen.head())


Columns in df_unseen:
['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment', 'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd', 'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt', 'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever', 'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
Unique values in MARITALSTATUS: ['Married' 'Single']
Unique values in EDUCATION: ['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS']
Unique values in GENDER: ['M' 'F']
Unique values in last_prod_enq2: ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']

In [116]:
# Check the value counts in the 'EDUCATION' column
if 'EDUCATION' in df_unseen.columns:
    print("Value counts in 'EDUCATION' column before transformation:")
    print(df_unseen['EDUCATION'].value_counts())

    # Transform 'EDUCATION' column
    education_mapping = {
        'SSC': 1,
        '12TH': 2,
        'GRADUATE': 3,
        'UNDER GRADUATE': 3,
        'POST-GRADUATE': 4,
        'OTHERS': 1,
        'PROFESSIONAL': 3
    }

    # Map values and handle missing values by filling them with 0
    df_unseen['EDUCATION'] = df_unseen['EDUCATION'].map(education_mapping)

    # Fill NaNs with 0 before converting to integer
    df_unseen['EDUCATION'] = df_unseen['EDUCATION'].fillna(0).astype(int)

    print("Value counts in 'EDUCATION' column after transformation:")
    print(df_unseen['EDUCATION'].value_counts())
else:
    print("Column 'EDUCATION' not found in df_unseen.")

# Print the DataFrame info
df_unseen.info()

Value counts in 'EDUCATION' column before transformation:
EDUCATION
0    100
Name: count, dtype: int64
Value counts in 'EDUCATION' column after transformation:
EDUCATION
0    100
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            100 non-null    float64
 1   pct_tl_closed_L6M          100 non-null    float64
 2   Tot_TL_closed_L12M         100 non-null    int64  
 3   pct_tl_closed_L12M         100 non-null    float64
 4   Tot_Missed_Pmnt            100 non-null    int64  
 5   CC_TL                      100 non-null    int64  
 6   Home_TL                    100 non-null    int64  
 7   PL_TL                      100 non-null    int64  
 8   Secured_TL                 100 non-null    int64  
 9   Unsecured_TL               100 non-null    int64  
 10  Other_TL      

In [117]:
# Perform one-hot encoding on the specified columns if they exist
categorical_columns = ['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

# Check if all categorical columns exist
missing_categorical_columns = [col for col in categorical_columns if col not in df_unseen.columns]
if missing_categorical_columns:
    print(f"Missing categorical columns: {missing_categorical_columns}")

# Perform one-hot encoding
df_encoded_unseen = pd.get_dummies(df_unseen, columns=[col for col in categorical_columns if col in df_unseen.columns])

# Print the DataFrame info
df_encoded_unseen.info()

# Describe the DataFrame
k = df_encoded_unseen.describe()
print(k)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               100 non-null    float64
 1   pct_tl_closed_L6M             100 non-null    float64
 2   Tot_TL_closed_L12M            100 non-null    int64  
 3   pct_tl_closed_L12M            100 non-null    float64
 4   Tot_Missed_Pmnt               100 non-null    int64  
 5   CC_TL                         100 non-null    int64  
 6   Home_TL                       100 non-null    int64  
 7   PL_TL                         100 non-null    int64  
 8   Secured_TL                    100 non-null    int64  
 9   Unsecured_TL                  100 non-null    int64  
 10  Other_TL                      100 non-null    int64  
 11  Age_Oldest_TL                 100 non-null    int64  
 12  Age_Newest_TL                 100 non-null    int64  
 13  time_s

In [121]:
expected_features = model.get_booster().feature_names

In [122]:
# Align features
for feature in expected_features:
    if feature not in df_encoded_unseen.columns:
        df_encoded_unseen[feature] = 0

# Reorder the columns to match the expected features
df_encoded_unseen = df_encoded_unseen[expected_features]

# Ensure no additional columns are present
df_encoded_unseen = df_encoded_unseen.loc[:, expected_features]

# Check the DataFrame info
df_encoded_unseen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 90 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    100 non-null    int64  
 1   Total_TL                      100 non-null    int64  
 2   Tot_Closed_TL                 100 non-null    int64  
 3   Tot_Active_TL                 100 non-null    int64  
 4   Total_TL_opened_L6M           100 non-null    int64  
 5   Tot_TL_closed_L6M             100 non-null    int64  
 6   pct_tl_open_L6M               100 non-null    float64
 7   pct_tl_closed_L6M             100 non-null    float64
 8   pct_active_tl                 100 non-null    int64  
 9   pct_closed_tl                 100 non-null    int64  
 10  Total_TL_opened_L12M          100 non-null    int64  
 11  Tot_TL_closed_L12M            100 non-null    int64  
 12  pct_tl_open_L12M              100 non-null    int64  
 13  pct_tl

In [133]:
for i in data_encoded.columns:
  if i not in list(df_encoded_unseen.columns):
    print(i)

Approved_Flag


In [134]:
model=xgb.XGBClassifier(objective='multi:softmax',  num_class=4,colsample_bytree=0.9,learning_rate=1,max_dep=3,alpha=10,n_estimators=100)
model.fit(x_train,y_train)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_dep=3, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, ...)

In [131]:
# Drop 'Approved_Flag' if it exists
if 'Approved_Flag' in df_encoded_unseen.columns:
    df_encoded_unseen = df_encoded_unseen.drop('Approved_Flag', axis=1)

# Predict on the unseen data
y_pred = model.predict(df_encoded_unseen)

# Add the predicted labels to the DataFrame

In [132]:
df_unseen['Target_variable']=y_pred

In [136]:
df_unseen.to_excel(r'/content/sample_submission.xlsx', index = False)

In [137]:
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 18427.24 seconds
